# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `predictRain(ilkoProb: int, deliverProbs: List[int]) -> float`

    
    Ilko is a meteorologist whose goal is to predict the weather for the report given in the evening news on TV.
    Ilko has a complex numerical model that tries to predict a single boolean: whether it will rain tomorrow.
    The success rate of the model's predictions is ilkoProb percent.
    Every day at 4:47 pm, Ilko runs the model, reads its output, thinks about it, and finally he sends a one-word instruction to the TV station: he either tells them to broadcast the word "sunny" or the word "rainy".

    However, the TV station does not simply broadcast the word Ilko sent.
    There is a whole chain of people who get involved:
    Ilko reports the forecast to his contact person, that person tells their secretary to pass it on, the secretary writes it down and uses internal mail to send it to a different floor, and so on.
    The problem is that each person in the chain will, with some probability, mix up the message and deliver the opposite forecast from the one they received.

    You are given the List[int] deliverProbs.
    Each element of deliverProbs corresponds to one person in the chain, in the order in which the message passes through them.
    Person i will correctly deliver the forecast they receive with probability deliverProbs[i] percent.

    Ilko knows all the information mentioned above, including all the probabilities.
    His reputation depends on the accuracy of the forecast that actually gets aired by the TV station.
    Thus, he will do all he can to maximize the accuracy of that forecast.

    Compute and return the probability that the forecast aired by the TV station tonight will match the weather tomorrow.

    Notes
    -Formally, assume that all random events mentioned in the statement are mutually independent. In particular, the probability that Ilko's model correctly predicts whether it will rain tomorrow is ilkoProb/100.
    -Your answer will be accepted if it has a relative or an absolute error up to 1e-9.
    -Remember that Ilko has perfect knowledge of all things mentioned in the problem statement, and that he is actively trying to maximize the probability that the weather will match the forecast broadcast by the station.
 
    Constraints
    -ilkoProb will be between 0 and 100, inclusive.
    -deliverProbs will have between 0 and 50 elements, inclusive.
    -Each element of deliverProbs will be between 0 and 100, inclusive.
 
    Examples
    0)
        93
    {}

    Returns: 0.93
    Ilko's model correctly predicts rain with probability 93%.
    There are no people involved in the TV station.
    Thus, what happens is that:

    Ilko looks at the prediction given by his model.
    Ilko sends that prediction to the TV station.
    The prediction goes straight into the broadcast.

    Obviously, the forecast aired on the news will match reality with probability 93%.

    1)
        93
    {50}

    Returns: 0.5
    Ilko uses the same model, but now he reports to a person who always loses it and then just flips a coin to determine what forecast the station will broadcast that night.

    2)
        100
    {90,90}

    Returns: 0.82

    In this test case, Ilko's model is completely accurate. Sadly, the forecast goes through two people before it gets aired, and each of them will make a mistake with probability 10%. That's still good enough to mostly preserve the forecast Ilko reports to the station, but they do introduce some mistakes here and there.

    Note that the actual probability that the station airs the correct forecast is 82%, not 81%.
    This is because the following is also a possible chain of events:

    Ilko reports the correct forecast to the station.
    The first person makes a mistake and reports the opposite forecast to the second person.
    The second person also makes a mistake and broadcasts the opposite forecast from the one they received - but that is the forecast Ilko originally sent!

    3)
        89
    {13, 92, 7}

    Returns: 0.7084846399999999

    4)
        50
    {3, 17, 92, 34, 2, 14}

    Returns: 0.5
    This time, the accuracy of Ilko's model is as good as a coin flip. The people at the TV station don't really matter, the final forecast will still be correct with probability exactly 50 percent.

    

Here, we need to calculate the probabilty that Ilko will get the right forcast.

In order to compute this probability, we need to compute the total probability of deliverProbs when they deliver the right/wrong message.
A message is right if 0 or an even number of mistakes are made, and wrong otherwise.
The idea is to keep an array with this state and multiply the appropriate probability (p if it is right and 1-p if it is wrong).

Note that there are up to 50 elements in deliverProbs, therefore I will need to compute up to $2^{50}$ different combinations, however I can stop whenever the relative error is $< 10^{-9}$ and return it early.

Lets start with a class to calculate the probability.

In [1]:
class Probability:
    def __init__(self, probs):
        self.probs = probs
        self.precision = 1e-14
        self.memory = dict()
        
    def calc(self, state):
        p = 1
        for i, s in enumerate(state):
            if s:
                p *= self.probs[i]
            else:
                p *= 100-self.probs[i+1]
            
            if p == 0:
                return 0
                
        return p/100**len(state)
        
P = Probability([50, 100, 100])
print(P.calc([1, 0, 1]))
print(P.calc([1, 1, 1]))

0
0.5


Looks like the class is working.

Now I will try to write a function to calculate all the possible combinations of passing the right/wrong forcast.
This function will keep track of even and odd combinations to decide is the pipeline of employees will pass the right or wrong message.
I will use memoization to return an early result if the probability is smaller than a given precision.

In [ ]:
import copy

def calculate_prob(deliverProbs, prob = [], memory = dict(), idx = 0):
    
    PRECISION = 1e-14
    
    if p is None:
        p = {
            'even': 1, 'odd': 1
        }
        
    if p['even'] < PRECISION and p['odd'] < PRECISION:
        p['even'] += p['even']*2**(len(deliverProbs-idx-2))
        p['odd']  += p['odd']*2**(len(deliverProbs-idx-2))
        return prob
    
    h = hash(str(prob))
    if h in memory.keys():
        return memory[h]
    
    if len(prob) < len(deliverProbs):
        new_prob1 = copy.copy(prob)
        new_p1    = copy.deepcopy(p)
        new_prob1.append(1)
        prob['even'] = 
    
    
    

The code is getting cumbersome, therefore I decided to stop before executing and completing it.

After thinking for some time, I realized that there is a much easier way to solve the problem:
since the probabilities are independent, I can always reduce two probabilities into a single one.

Consider the case of three probabilities [p1, p2, p3].
- the probability of the first two people getting it right <code> right = p1 * p2+(1-p1) * (1-p2) </code>
- since the probabilities are independent, we now have a new problem with [right, p3]
- hence the final probability of getting it right is <code> right * p3+(1-right) * (1-p3) </code>

Lets implement the function to calcute this.

In [2]:
def get_right(deliverProbs):
    
    if len(deliverProbs) == 0:
        return 1
    elif len(deliverProbs) == 1:
        return deliverProbs[0]
    
    p1 = deliverProbs[0]/100
    p2 = deliverProbs[1]/100
    right = p1*p2+(1-p1)*(1-p2)
    
    for p in deliverProbs[2:]:
        right = right*p+(1-right)*(1-p)
        
    return right

get_right([90, 90])

0.8200000000000001

In [3]:
get_right([13, 92, 7])

-3.5404

The first result is correct, while the second is incorrect.

I forgot to divide by 100 in the for loop.

In [4]:
def get_right(deliverProbs):
    
    if len(deliverProbs) == 0:
        return 1
    elif len(deliverProbs) == 1:
        return deliverProbs[0]
    
    p1 = deliverProbs[0]/100
    p2 = deliverProbs[1]/100
    right = p1*p2+(1-p1)*(1-p2)
    
    for p in deliverProbs[2:]:
        right = right*p/100+(1-right)*(1-p/100)
        
    return right

get_right([13, 92, 7])

0.767288

The probability looks correct now.

If the probability of the model is 0.89, than the final probability of getting it right is

In [5]:
0.767288*0.89+(1-0.767288)*(1-0.89)

0.7084846399999999

This is the correct result.

Let me implent the function predictRain and a few tests.

In [6]:
from typing import List
from numpy import isclose

def get_right(deliverProbs):
    
    if len(deliverProbs) == 0:
        return 1
    elif len(deliverProbs) == 1:
        return deliverProbs[0]
    
    p1 = deliverProbs[0]/100
    p2 = deliverProbs[1]/100
    right = p1*p2+(1-p1)*(1-p2)
    
    for p in deliverProbs[2:]:
        right = right*p/100+(1-right)*(1-p/100)
        
    return right

def predictRain(ilkoProb: int, deliverProbs: List[int]) -> float:
    p = get_right(deliverProbs)
    ilkoProb /= 100
    
    return ilkoProb*p+(1-ilkoProb)*(1-p)

def test():
    params = [
        [93, [50], 0.5],
        [100, [90,90], 0.82],
        [89, [13, 92, 7], 0.7084846399999999],
        [50, [3, 17, 92, 34, 2, 14], 0.5],
    ]
    for p in params:
        res = predictRain(p[0], p[1])
        print(p, res, p[2])
        assert isclose(res, p[2], rtol=1e-10, atol=1e-10)
        
test()

[93, [50], 0.5] 43.07 0.5


AssertionError: 

I am failing the first assertion. The result above 1 indicates that I forgot to divide deliverProbs by 100 somewhere.

The problem happend when len(deliverProbs) == 1.
In this case I should return the result divided by 100.

In [7]:
from typing import List
from numpy import isclose

def get_right(deliverProbs):
    
    if len(deliverProbs) == 0:
        return 1
    elif len(deliverProbs) == 1:
        return deliverProbs[0]/100
    
    p1 = deliverProbs[0]/100
    p2 = deliverProbs[1]/100
    right = p1*p2+(1-p1)*(1-p2)
    
    for p in deliverProbs[2:]:
        right = right*p/100+(1-right)*(1-p/100)
        
    return right

def predictRain(ilkoProb: int, deliverProbs: List[int]) -> float:
    p = get_right(deliverProbs)
    ilkoProb /= 100
    
    return ilkoProb*p+(1-ilkoProb)*(1-p)

def test():
    params = [
        [93, [50], 0.5],
        [100, [90,90], 0.82],
        [89, [13, 92, 7], 0.7084846399999999],
        [50, [3, 17, 92, 34, 2, 14], 0.5],
    ]
    for p in params:
        res = predictRain(p[0], p[1])
        print(p, res, p[2])
        assert isclose(res, p[2], rtol=1e-10, atol=1e-10)
    print('done')
    
test()

[93, [50], 0.5] 0.5 0.5
[100, [90, 90], 0.82] 0.8200000000000001 0.82
[89, [13, 92, 7], 0.7084846399999999] 0.7084846399999999 0.7084846399999999
[50, [3, 17, 92, 34, 2, 14], 0.5] 0.5 0.5
done


Nice, all tests are working. 

Let me add a few more

In [13]:
def test():
    params = [
        [93, [50], 0.5],
        [100, [90,90], 0.82],
        [89, [13, 92, 7], 0.7084846399999999],
        [50, [3, 17, 92, 34, 2, 14], 0.5],
        [50, [3, 17, 92, 34, 2, 14], 0.5],
        [50, [50]*50, 0.5],
        [0, [100]*50, 0],
        [0, [0]*49, 1],
        [0, [100, 90], 0.1],
    ]
    for p in params:
        res = predictRain(p[0], p[1])
        print(p, res, p[2])
        assert isclose(res, p[2], rtol=1e-10, atol=1e-10)
    print('done')
    
test()

[93, [50], 0.5] 0.5 0.5
[100, [90, 90], 0.82] 0.8200000000000001 0.82
[89, [13, 92, 7], 0.7084846399999999] 0.7084846399999999 0.7084846399999999
[50, [3, 17, 92, 34, 2, 14], 0.5] 0.5 0.5
[50, [3, 17, 92, 34, 2, 14], 0.5] 0.5 0.5
[50, [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50], 0.5] 0.5 0.5
[0, [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100], 0] 0.0 0
[0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1] 1.0 1
[0, [100, 90], 0.1] 0.09999999999999998 0.1
done
